In [1]:
!pip install -q langchain==0.0.219 openai==0.27.8

# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

!pip install -q pandas==1.5.3
!pip install -q unstructured==0.7.12
!pip install -q wikipedia==1.4.0
!pip install -q pypdf==3.12.0
!pip install -q jq==1.4.1
!pip install -q nltk==3.8.1
!pip install -q tiktoken==0.4.0
!pip install -q transformers==4.29.2 sentencepiece==0.1.99
!pip install -q sentence-transformers==2.2.2
!pip install -q cohere==4.11.2
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 22.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 31.4 MB/s eta 0

In [5]:
from dotenv import dotenv_values
env_values = dotenv_values("./app.env")
openai_api_key = env_values['OPEN_API_KEY']

In [6]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key,
             model_name="gpt-3.5-turbo",
             temperature=0.5)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Loader

## Pandas

In [7]:
import pandas as pd
from langchain.document_loaders import DataFrameLoader



In [8]:
!wget -O youtube-sub.csv https://raw.githubusercontent.com/Petlja/JupyterBookSrCyr/master/podaci/Top%2025%20YouTubers.csv

--2024-06-17 21:34:42--  https://raw.githubusercontent.com/Petlja/JupyterBookSrCyr/master/podaci/Top%2025%20YouTubers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1153 (1.1K) [text/plain]
Saving to: ‘youtube-sub.csv’

youtube-sub.csv     100%[===================>]   1.13K  --.-KB/s    in 0s      

2024-06-17 21:34:42 (12.6 MB/s) - ‘youtube-sub.csv’ saved [1153/1153]



In [9]:
data =  pd.read_csv("youtube-sub.csv",sep = ",")
data.head()

,RANK,GRADE,NAME,VIDEOS,SUBSCRIBERS,VIEWES
0,1,A++,T-Series,13629,105783888,76945588449
1,2,A,PewDiePie,3898,97853589,22298927681
2,3,A+,5-Minute Crafts,3341,58629572,14860695079
3,4,A++,Cocomelon - Nursery Rhymes,441,53163816,33519273951
4,5,A++,SET India,31923,51784081,36464793233


In [11]:
data = data[ ["NAME", "VIDEOS", "SUBSCRIBERS", "VIEWES"] ]
data.head()

,NAME,VIDEOS,SUBSCRIBERS,VIEWES
0,T-Series,13629,105783888,76945588449
1,PewDiePie,3898,97853589,22298927681
2,5-Minute Crafts,3341,58629572,14860695079
3,Cocomelon - Nursery Rhymes,441,53163816,33519273951
4,SET India,31923,51784081,36464793233


In [12]:
# we need to define the main column
llm_loader =  DataFrameLoader(data,page_content_column= "NAME")
llm_loader = llm_loader.load()

In [13]:
llm_loader[0]

Document(page_content='T-Series', metadata={'VIDEOS': 13629, 'SUBSCRIBERS': 105783888, 'VIEWES': 76945588449})

## URLS

In [15]:
from langchain.document_loaders import WebBaseLoader , UnstructuredURLLoader
urls = [
    "https://www.apple.com/in/support/products/faqs.html",
    "https://www.apple.com/legal/sales-support/",
]


In [16]:
llm_loader = UnstructuredURLLoader(urls)
llm_loader = llm_loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [17]:
llm_loader[0]

Document(page_content='Apple\n\nStoreMaciPadiPhoneWatchAirPodsTV & HomeEntertainmentAccessoriesSupport\n\n\n\n0+\n\nAppleCare Products\n\nOpen Menu\n\nClose Menu\n\nMac\n\niPad\n\niPhone\n\nApple TV\n\niPod\n\nBuy Now\n\nFrequently Asked Questions\n\nWhere can I purchase the AppleCare Protection Plan?\n\nThe AppleCare Protection Plan is available at the Apple Online Store and many Apple-authorised resellers and wireless service providers.\n\nHow do I initiate repair service under the AppleCare Protection Plan?\n\nCarry-in service. Carry your product into an Apple Authorised Service Provider.\n\nOnsite service. Contact us and we’ll help you arrange an Apple-authorised repair for your desktop Mac at your location, at no additional charge.\n\nDo-It-Yourself service. Contact us and we may be able to send you what you need to service your own product, such as accessories.\n\nThe AppleCare Protection Plan for Mac, for iPod and for Apple TV provide global repair coverage. Service will be limi

## Wikibedia

In [18]:
from langchain.document_loaders import WikipediaLoader

llm_loader =  WikipediaLoader(query="Apple")
llm_loader = llm_loader.load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [19]:
len(llm_loader)

9

In [20]:
llm_loader[0]

Document(page_content='An apple is a round, edible fruit produced by an apple tree (Malus spp., among them the domestic or orchard apple; Malus domestica). Apple trees are cultivated worldwide and are the most widely grown species in the genus Malus. The tree originated in Central Asia, where its wild ancestor, Malus sieversii, is still found. Apples have been grown for thousands of years in Eurasia and were introduced to North America by European colonists. Apples have religious and mythological significance in many cultures, including Norse, Greek, and European Christian tradition.\nApples grown from seed tend to be very different from those of their parents, and the resultant fruit frequently lacks desired characteristics. For commercial purposes, including botanical evaluation, apple cultivars are propagated by clonal grafting onto rootstocks. Apple trees grown without rootstocks tend to be larger and much slower to fruit after planting. Rootstocks are used to control the speed of 

## PDF

In [21]:
!wget -O laravel.pdf https://laraveldaily.com/wp-content/uploads/2020/04/laravel-tips-2020-04.pdf

--2024-06-17 21:46:00--  https://laraveldaily.com/wp-content/uploads/2020/04/laravel-tips-2020-04.pdf
Resolving laraveldaily.com (laraveldaily.com)... 206.189.51.206
Connecting to laraveldaily.com (laraveldaily.com)|206.189.51.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314570 (307K) [application/pdf]
Saving to: ‘laravel.pdf’

laravel.pdf         100%[===================>] 307.20K   774KB/s    in 0.4s    

2024-06-17 21:46:01 (774 KB/s) - ‘laravel.pdf’ saved [314570/314570]



In [22]:
from langchain.document_loaders import PyPDFLoader

llm_loader = PyPDFLoader("laravel.pdf")
llm_loader = llm_loader.load()

In [24]:
len(llm_loader)

41

In [25]:
llm_loader[0]

Document(page_content=' \n \n100 (and counting)\n \nLaravel Quick Tips\n \n \n \nPrepared by:\n \nPovilas Korop / LaravelDaily Team\n \nwww.laraveldaily.com\n \n \npovilas@laraveldaily.com\n \n \nLast updated:\n \n \nApril 2020\n \n \n \n \n \n \n \nSupport our work by checking our Laravel adminpanel generator: \n\u200b\nwww.quickadminpanel.com\n ', metadata={'source': 'laravel.pdf', 'page': 0})

# There are different ways to load our data and there are many methoods in documentation

# Chunking

In [26]:
from langchain.document_loaders import WikipediaLoader

query_1 = "Nikola Tesla"
query_2 = "Thomas Edison"

# ==============
docs_1 = WikipediaLoader(query=query_1, load_max_docs=1,
                         doc_content_chars_max=20_000).load()

text_document_1 = docs_1[0].page_content

# ==============
docs_2 = WikipediaLoader(query=query_2, load_max_docs=1,
                         doc_content_chars_max=20_000).load()

text_document_2 = docs_2[0].page_content


In [27]:
documents = [ text_document_1, text_document_2]
metadatas = [ {"document":query_1}, {"document":query_2} ]

## Textsplitter

In [29]:
from langchain.text_splitter import CharacterTextSplitter

# chunk_size >>>>  number of characters in each chunks
# chunck overlap >>>>  number of charcter mutual between two chunks
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.create_documents(documents ,metadatas =  metadatas)


In [30]:
len(texts)

31

In [31]:
texts[0]

Document(page_content="Nikola Tesla (; Serbian Cyrillic: Никола Тесла, [nǐkola têsla]; 10 July [O.S. 28 June] 1856 – 7 January 1943) was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase

## NLTK

In [32]:
# NLTK is better because working on chunking when the word or sentence done
from langchain.text_splitter import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.create_documents(documents ,metadatas =  metadatas)

In [33]:
len(texts)

276

In [34]:
texts[0]

Document(page_content='Nikola Tesla (; Serbian Cyrillic: Никола Тесла, [nǐkola têsla]; 10 July [O.S.', metadata={'document': 'Nikola Tesla'})

## TikToken

In [38]:
from langchain.text_splitter import TokenTextSplitter

text_splitter =  TokenTextSplitter(chunk_size = 100,chunk_overlap = 10 )

texts =  text_splitter.create_documents(documents,metadatas = metadatas)

In [39]:
len(texts)

92

In [40]:
texts[0]

Document(page_content='Nikola Tesla (; Serbian Cyrillic: Никола Тесла, [nǐkola têsla]; 10 July [O.S. 28 June] 1856 – 7 January 1943) was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire,', metadata={'document': 'Nikola Tesla'})

## Transformers Token

In [42]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer,
    chunk_size=100,
    chunk_overlap=20,
)


In [43]:
texts =  text_splitter.create_documents(documents,metadatas = metadatas)

In [44]:
len(texts)

30

In [45]:
texts[0]

Document(page_content="Nikola Tesla (; Serbian Cyrillic: Никола Тесла, [nǐkola têsla]; 10 July [O.S. 28 June] 1856 – 7 January 1943) was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase